# CloudTrail Summary

This Notebook reads evenst from CloudTrail and presents for each IAM Role a summary of the usage, i.e., a breakdown by IAM Role, EventSource (AWS Service), EventName (API Method), and ErrorCode.

## Usage:

- Install dependencies (see below)
- (Optionally) set desired default in the settings cell
- Run all cells in this notebook (menu Run, Run All Cells)
- Select the desired AWS Profile
- Select the desired Regions
- Select the number of events in each region
- Click `[ Fetch Events ]`
- Wait for the events to be fetched
- See the results in the table (pandas dataframe)

## Imports

Here we import the modules we need, make sure the following dependencies have been installed: `boto3` and `pandas`.

In [8]:
import ipywidgets as widgets
import json
import pandas as pd
import boto3

## Settings

In the following cell, you can set default values for the profile and regions selections.
You should preferably include `us-east-1` in the region selection, as some (IAM) events are only logged there.

In [9]:
default_profile = 'default'
default_regions = [ 'us-east-1', 'eu-west-1' ]

## AWS Profile and Region Selection

Here we have code to select the desired AWS Profile and Region(s).
The `sessions` dictionary will contain for each selected region, with the region name as key, a boto3 session with the selected profile.
With each change the boto3 session(s) will be automatically updated.

In [10]:
sessions = {}
current_profile = None

session_output = widgets.Output()

profile_dropdown = widgets.Dropdown(
    options=boto3.Session().available_profiles,
    description='AWS Profile',
    value=default_profile,
)

region_checkboxes = {
    region_name: widgets.Checkbox(
        description=region_name,
        value=region_name in default_regions,
    )
    for region_name in boto3.Session().get_available_regions('cloudtrail')
}


def create_sessions(*args, **kwargs):
    global sessions, current_profile
    
    profile_name = profile_dropdown.value

    for region_name, checkbox in region_checkboxes.items():
        if checkbox.value:
            if not current_profile == profile_name or region_name not in sessions:
                sessions[region_name] = boto3.session.Session(profile_name=profile_name, region_name=region_name)
        else:
            if region_name in sessions:
                del sessions[region_name]

    current_profile = profile_name
    with session_output:
        session_output.clear_output()
        if len(sessions) > 0:
            try:
                sts = list(sessions.values())[0].client('sts')
                print(f"Identity: {sts.get_caller_identity().get('Arn')}")
            except BaseException as err:
                sessions = {}
                print(err)
        else:
            print("No Sessions")

for widget in [profile_dropdown, *region_checkboxes.values()]:
    widget.observe(create_sessions, names='value')

display(
    widgets.VBox([
        profile_dropdown,
        widgets.GridBox(
            tuple(region_checkboxes.values()),
            layout=widgets.Layout(grid_template_columns="repeat(4, 120px)")),
        session_output,
    ])
)

create_sessions()

## CloudTrail Summary

Here we have code to query CloudTrail and present a summary of the found events per IAM Role.

Using the slider, select the number of events, per selected region, you wish to fetch, then click `[ Fetch Events ]`. Now the desired amount of events will be fetched from CloudTrail in each selected region. The `events` variable will be populated with a Pandas DataFrame containing all fetched events. A summary of the events will be displayed.

In [11]:
events = pd.DataFrame()
summary_output = widgets.Output()
progress = widgets.IntProgress(description='Loading', value=0, min=0, max=0)
progress.layout.visibility = 'hidden'


def load_events(query=[], pages=10, pagesize=50):
    global events
    
    def update_progress(*args, **kwargs):
        progress.max = pages * len(sessions)
        progress.value += 1
        return True
    
    events = pd.concat([events] +[
        page_df.drop(['CloudTrailEvent'], axis=1).join(pd.json_normalize(page_df['CloudTrailEvent'].apply(json.loads)))

        for session in sessions.values()

        for client in [session.client('cloudtrail')]

        for page in client.get_paginator('lookup_events').paginate(
            PaginationConfig={
                'MaxItems': pages * pagesize,
                'PageSize': pagesize,
            },
            LookupAttributes=query,
        )

        if update_progress()

        for page_df in [pd.DataFrame(page['Events'])]
        
        if 'CloudTrailEvent' in page_df
    ])

    # drop duplicates
    events = events.loc[events.astype(str).drop_duplicates().index]
    
    return events


def summarize(df):
    if 'userIdentity.sessionContext.sessionIssuer.arn' not in df:
        return pd.DataFrame()
    
    summary_cols = ['userIdentity.sessionContext.sessionIssuer.arn', 'EventSource', 'EventName', 'errorCode']
    return df[~df['userIdentity.sessionContext.sessionIssuer.arn'].isnull()].reindex([*summary_cols, 'eventTime'], axis="columns").drop_duplicates().fillna('-').groupby(summary_cols).max().sort_values([*summary_cols, 'eventTime'])



load_pages = widgets.IntSlider(
    value=500,
    min=50,
    max=4000,
    step=50,
    description='Events',
    readout=True,
    readout_format='d'
)

load_button = widgets.Button(description='Fetch Events')
def load(*args):
    summary_output.clear_output()
    progress.value = 0
    progress.layout.visibility = 'visible'
    events = load_events([], load_pages.value/50, 50)
    with summary_output:
        display(summarize(events))
    progress.layout.visibility = 'hidden'
load_button.on_click(load)

pd.set_option('display.max_rows', None)

display(
    widgets.VBox([
        widgets.HBox([load_pages, load_button, progress]),
        summary_output,
    ])
)